## DATASET CREATION

Unfortunately, the dataset I will work with does not come as one already done dataset - as it is mostly usual in Data Science projects. <br /> 
To get the sheer experience of how a normal Data Science job looks like I also wanted to dive into this issue. By experience I can tell that this takes a shit ton of time - like almost 80% - of preparing the data. <br /> 
Since the data usually shares the same countries and lists mostly the same years of the recording I used this as a primary key where I connect the datapoints with each other. <br /> 
However, the  countries are listed in rows along with the year of the recording - I want to have a final dataset that looks like follows: <br /> 

|Country | Afghanistan | Albania | ... | Zimbabwe | 
| ----- | ----------  | ------ | ----- | ------- | 
|Alcohol consumption [l] | 0.2 | 2.4 | ... | 0.01 |
|Human Develpment Index (HDI) | 0.1 | 0.15 | ... | 0.1 | 
|... | ... | ... | ... | ... |
|Healthcare Expenditure [$] | 13.322 | 15.211 | ... | 1.039 |

Thus, I have to transpose each of the countries and record each of the years as seperate entry in the dataset. <br /> 

All the data is [publicly available](https://ourworldindata.org), and this source is trusted by many notorious companies such as Vox, The Ney York times and even the top universities of this world like MIT, Oxford, Stanford. <br /> 
Hence, I assume that this data is rather based on actual recording from the respective country. <br /> 
Even the United Nation published their records in this page and I bet that these folks do some amazing work, which we can trust. <br />

One more thing to mention with respect to the countries is that some of them were only listed together such as *Serbia and Montenegro*, *Belgium Luxembourg*, ... and thus sometimes there is no single value for these countries available due to their combined listing. I decided not to give the credits only one specific country due to the unproportionality in their population and possibly cultural changes. <br /> 

But now let's not waste too much with the explaination part and go straight into how I merged the datafiles to one huge on. <br /> 

In [6]:
import os 
import sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

Do the preprocessing necessities with renaming the columns and dropping the ones we are not interested in, i.e. continents aggregated stuff, some islands and countries I have never heard of, etc. <br /> 
After doing that, we save the new csv file again.  <br /> 
Moreover, I also delete all the entries of 1979 and before to not have too much history data, which speeds up the computation by a looooot at least 1 day of computation saved due to that! <br /> 

In [7]:
def rename_Countries_drop_Unnecessary(df, name): 
    print("Before {} shape: {}".format(name, df.shape))
    # Input is only the dataframe with the country names in the column 'Entity'
    right_names = []
    for data in df['Entity']:
        ## RENAMING COUNTRIES FOR DATA CONSISTENCY
        if 'Hong Kong' in data:
            data = 'Hong Kong'
        if 'Taiwan' in data: 
            data = 'Taiwan'
        if 'Macao' in data: 
            data = 'Macao'
        if 'Ethiopia' in data: 
            data = 'Ethiopia'
        if 'Sudan' in data: 
            data = 'Sudan'        
        if 'Czechia' in data: 
            data = 'Czech Republic'
        if 'Syria' in data: 
            data = 'Syria'
        if 'Russia' in data: 
            data = 'Russia'
        if "Ivoire" in data:
            data = "Cote d'Ivoire"

        # America 
        if 'US' in data: 
            data = 'United States'
        if 'USA' in data: 
            data = 'United States'
        if 'U.S.A.' in data: 
            data = 'United States'
        if 'U.S.A' in data: 
            data = 'United States'
        if 'United States of' in data: # gets United States of America
            data = 'United States'
        if data == 'America': 
            data = 'United States'
        right_names.append(data) 
        
    ## Replace the names with the consistent names of them
    right_names = pd.Series(right_names)
    df['Entity'] = right_names

    ## DELETE THE ENTRIES WHICH ARE NOT IN OUR MASTER COUNTRY LIST
    countries = list(set(df["Entity"]))
    for country in countries: 
        if country not in interested_countries:
            idx = list(df['Entity']).index(country)
            endCountry = idx + list(df['Entity']).count(country)
            ranges = np.arange( idx , endCountry )  
            df.drop(df.index[ranges], inplace = True)
            
    ## DELETE ENTRIES BEFORE 1980
    df = df[df["Year"] > 1979]
    
    ## SAVE THE FILE REDUCED AND CHANGED NAME AGAIN
    print("After: {}".format(df.shape))
    df.to_csv(os.path.join(datapath, name))

Define the countries we want to have in our Masterlist and thus in the dataset.

In [8]:
interested_countries = ['Sweden', 'Norway', 'Finland', 'Iceland', 'Germany', 'Netherlands', 'Belgium', 'Luxembourg',
                        'England', 'Scotland', 'Wales', 'Ireland', 'United Kingdom', 'Switzerland', 'Austria', 'France',
                        'Italy', 'Spain', 'Portugal', 'Morocco', 'Tunisia', 'Egypt', 'Liechtenstein', 'Cyprus', 'Vatican',
                        'Kosovo', 'Serbia', 'Georgia', 'Greenland', 'Antigua and Barbuda', 'Hungary', 'Monaco', 'Israel',
                        'Albania', 'Iraq', 'Iran', 'Syria', 'Turkey', 'Palestine', 'Montenegro', 'Latvia', 'Jordan',
                        'Croatia', 'New Zealand', 'Eritrea', 'Libya', 'Belarus', 'Slovenia', 'Greece', 'Lithuania',
                        'Liberia', 'Slovakia', 'Estonia', 'Poland', 'Czech Republic', 'Armenia', 'Denmark', 'Bulgaria',
                        
                        'Russia', 'United States', 'Canada', 'Qatar', 'Kuwait', 'Mexico', 'South Africa', 'Fiji', 'Oman',
                        'Japan', 'United Arab Emirates', 'South Korea', 'Macao', 'Hong Kong', 'China', 'Thailand', 'Belize',
                        'Taiwan', 'Vietnam', 'Malaysia', 'Indonesia', 'India', 'Philippines', 'Australia', 'Laos', 'Bhutan',
                        
                        'Kyrgyzstan', 'Kazakhstan', 'Uzbekistan', 'Turkmenistan', 'Tajikistan', 'Pakistan', 'Afghanistan',
                        'Argentina', 'Brazil', 'Chile', 'Venezuela', 'Peru', 'Colombia', 'Guyana', 'Mauritius', 'Barbados', 
                        'Cuba', 'Panama', 'Bahamas', 'Puerto Rico', 'Costa Rica', 'Solomon Islands',  'Marshall Islands',
                        'Ecuador', 'Benin', 'Seychelles', 'Bolivia', 'Madagascar',  'Mauritania', 'Bosnia and Herzegovina', 
                        'Jamaica', 'Lebanon', 'Senegal', 'Malta', 'French Polynesia', 'Bahrain', 'Burundi', 'Swaziland',
                        'Tanzania', 'Central African Republic', 'Malawi', 'Djibouti', 'Mozambique', 'Macedonia', 'Sierra Leone',
                        'Democratic Republic of Congo', 'Namibia', 'Algeria', 'Trinidad and Tobago', "Cote d'Ivoire",
                         
                        'Samoa', 'Bermuda', 'Aruba', 'Myanmar', 'Cape Verde', 'Uganda', 'Togo', 'Guinea', 
                        'San Marino', 'Ukraine', 'North Korea', 'Papua New Guinea', 'Haiti', 'Ghana', 'Sudan',
                        'Faeroe Islands', 'Cambodia', 'Somalia',  'Kiribati', 'Tonga', 'Mongolia', 'Rwanda', 'Bangladesh',
                        'Suriname', 'Nauru', 'Zambia', 'Azerbaijan',  'Sri Lanka', 'Nigeria', 'Kenya', 'Comoros', 'Andorra', 
                        'Tuvalu', 'Zimbabwe', 'Yemen', 'Cameroon', 'El Salvador', 'Angola', 'Curacao', 'Nicaragua',
                        'Saudi Arabia', 'Lesotho', 'Moldova', 'Gabon', 'Grenada', 'Mali', 'Romania', 'Guatemala', 'Dominican Republic', 
                        'Honduras', 'Congo',  'Burkina Faso',  'Saint Lucia', 'Cayman Islands', 'Botswana', 'Ethiopia', 
                        'Chad', 'Uruguay', 'Maldives', 'Gibraltar', 'Paraguay', 'Niger', 'Nepal']

In [9]:
# Define the path where we have our data stored and want to have it stored as well.
datapath = os.path.join(os.path.join(os.getcwd(), 'data'), 'Health')
datapath

'C:\\Users\\Lenny\\Documents\\Studium_Robotics (M.Sc.)\\03_Semester 3 - Oslo ERASMUS\\01_Applied Data Analysis and Machine Learning\\Project 3\\data\\Health'

Call each single file in our data directory and process each one according to the rules we set previously (Renaming and Deleting entries). 

In [18]:
dataFileNames = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))]
type4Cols = []
type7Cols = []
manualLists = []

# the datasets mostly have the same size of 4 columns and same setup so let's get those first

for file in dataFileNames: 
    try:
        df = pd.read_csv(os.path.join(datapath, str(file) ) , encoding='latin-1')
    except: 
        print("problems with this guy: {}".format(file))
        manualLists.append(file)
    if df.shape[1] == 5: # one type of files (4 columns)
        type4Cols.append(file) 
        rename_Countries_drop_Unnecessary(df, str(file))
        
    elif df.shape[1] == 7: 
        type7Cols.append(file) 
    else: # manual shit  to do then 
        manualLists.append(file)

Before agricultural-area-per-capita.csv shape: (7753, 5)
After: (5150, 5)
Before alcohol-attributable-fraction-of-mortality.csv shape: (173, 5)
After: (173, 5)
Before annual-healthcare-expenditure-per-capita.csv shape: (3460, 5)
After: (3460, 5)
Before average-height-of-men-for-selected-countries.csv shape: (1203, 5)
After: (91, 5)
Before beer-consumption-per-person.csv shape: (8268, 5)
After: (5556, 5)
Before cancer-death-rates.csv shape: (5068, 5)
After: (5068, 5)
Before cardiovascular-disease-death-rates.csv shape: (5068, 5)
After: (5068, 5)
Before child-mortality.csv shape: (12383, 5)
After: (6434, 5)
Before co-emissions-per-capita.csv shape: (39453, 5)
After: (6878, 5)
Before consumption-per-smoker-per-day.csv shape: (5742, 5)
After: (5742, 5)
Before dementia-death-rates.csv shape: (5068, 5)
After: (5068, 5)
Before diarrheal-disease-death-rates.csv shape: (5068, 5)
After: (5068, 5)
Before fish-and-seafood-consumption-per-capita.csv shape: (7714, 5)
After: (5130, 5)
Before fruit-co

Set up the final dataframe which we are going to use in the Analysis part. <br /> 
Notice that I set it up with 5mio rows, however this is just to ensure that all the data will be safely stored in it. I will delete the empty rows after the dataset is created. <br /> 
So, it's just a placeholder until now and serves the purpose of not running into index errors/ too small row size.

In [11]:
# create dataframe where we want to paste everything inside
# 5.000.000 rows to not run into some problems while adding rows - delete later the other ones
final_df = pd.DataFrame(data = np.zeros( (5000, len(interested_countries)) ), 
                        index = np.arange(5000),
                        columns = [ country for country in interested_countries])
# save indices as strings to get meaningful names
final_df.index = final_df.index.map(str)
final_df.shape

(5000, 197)

Just a Testbench for getting the values I want

In [ ]:
# testing to get values of the specific entries and Co
columns = list(df.columns)
b = df[df['Entity'] == 'France']
c = b[b['Year'] == 1999]#[columns[-1]]
d = b[b['Year'] == 1999][columns[-1]].sum()
print(d)
c
#df.groupby(["Entity", 'Year']).describe()

The Magic happens down here. <br /> 
We loop through every preprocessed list, <br /> 
In each list we loop through every country and further <br /> 
we also iterate over each year in that country. <br /> 
There I use the Year and the name of the file/list to create an index name. In this index name we paste the respective country and its value in it. <br /> 
We do this for all the preprocessed lists, which takes a shit ton of time. <br /> 


In [ ]:
final_df[2650:2800]

In [26]:
%%time
# paste the values into the final_df from each single list
nextListIndex = 810

try: 
    for lists in type4Cols:
        if type4Cols.index('number-of-nurses.csv') >= type4Cols.index(lists):
            continue
        
        ## Read the file and get the all countries along with their reported years
        print("{} list out of {}, Index: {}, Name: {}".format(type4Cols.index(lists), len(type4Cols), nextListIndex, lists))
        df = pd.read_csv(os.path.join(datapath, str(lists) ) , encoding='latin-1')
        
        # get the col names, unique countries and unique years
        columns = list(df.columns)
        countries = list(set(df["Entity"]))
        years = list(set(df['Year']))
        
        # get a list of all the index/row names
        indexNamesArr = final_df.index.values

        ## groupby countries and then years accordingly
        #df.groupby(["Entity", 'Year'])

        firstListRun = False # flag for renaming the indices

        # loop thru every country in the list
        for country in countries:
            
            ## check if country is in our masterlist
            if country not in interested_countries: 
                # skip this item
                print("\tCountry: {} not in list - but we skip it.".format(country))
                continue
            
            # take a dataframe for one country at a time
            country_df = df[df['Entity'] == country]

            # loop thru every year within that country - assuming the years are in the same order for every country
            for year in years:
                
                # rename the indices only if it is the very first run for the country
                if not firstListRun:
                    indexName = str(columns[-1]) + ' in ' + str(year)
                    indexNamesArr[nextListIndex] = indexName
                    nextListIndex += 1
                    if nextListIndex % 20 == 0: 
                        print("\t\t" + str(indexName))

                ## get the proper value and fill empty ones, if not available, fill it with 0.000
                # note: .sum() is only having one element anyway, just done to get the value as a float not an array
                value = country_df[country_df["Year"] == year][columns[-1]].sum() if not country_df[country_df["Year"] == year][columns[-1]].empty else 0.000
                # get the name of the row
                idxName = str(columns[-1]) + ' in ' + str(year)
                
                #print("Country: {} found in the dataset at spot: {}".format(country, interested_countries.index(country)))
                
                # assign the value in the merged df with the value 
                final_df.iat[list(final_df.index.values).index(idxName), interested_countries.index(country)] = value # .iat[row, col]

            ## get the proper index for the next list to begin with   
            firstListRun = True
            # print progress
            if countries.index(country) % 60 == 0:
                print("\tWorking on country: {} out of {}".format(countries.index(country), len(countries)))
            
except Exception as e:
    print("Next List Index in line: {} of {}, Matrix Size: {}, list: {}, country: {}".format(i, nextListIndex, final_df.shape[0], lists, country))
    print(e)
    sys.exit()

32 list out of 62, Index: 810, Name: per-capita-egg-consumption-kilograms-per-year.csv
		Egg consumption per capita [kg] in 1989
		Egg consumption per capita [kg] in 2009
	Working on country: 0 out of 161
	Working on country: 60 out of 161
	Working on country: 120 out of 161
33 list out of 62, Index: 844, Name: per-capita-milk-consumption.csv
		Milk consumption per capita - Excluding Butter [kg] in 1995
	Working on country: 0 out of 161
	Working on country: 60 out of 161
	Working on country: 120 out of 161
34 list out of 62, Index: 878, Name: pisa-test-score-mean-performance-on-the-mathematics-scale.csv
		PISA [Mean performance on the mathematics scale] in 2003
	Working on country: 0 out of 74
	Working on country: 60 out of 74
35 list out of 62, Index: 884, Name: pisa-test-score-mean-performance-on-the-science-scale.csv
	Working on country: 0 out of 74
	Working on country: 60 out of 74
36 list out of 62, Index: 890, Name: population-density.csv
		Population density [people / km^2] in 1

In [35]:
lastFilledItem = list(final_df.index).index(indexName) + 1
final_df.drop(final_df.index[ np.arange(lastFilledItem, final_df.shape[0]) ], inplace=True)
final_df.shape

(1515, 197)

Finally save our sweet dataframe! **HURRRRRAAAAAAAAYY**

In [36]:
final_df.to_csv(os.path.join(datapath, "Merged UN Data 1980+.csv"))

BELOW HERE IS JUST THE PLAYGROUND TO TRY SOME STUFF OUT -> The dataset should be done in the top part

In [5]:
# open my baby again 
df = pd.read_csv(os.path.join(datapath, "Merged UN Data.csv"))

C:\Users\Lenny\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df[3550:3600]

,Unnamed: 0,Sweden,Norway,Finland,Iceland,Germany,Netherlands,Belgium,Luxembourg,England,...,Cayman Islands,Botswana,Ethiopia,Chad,Uruguay,Maldives,Gibraltar,Paraguay,Niger,Nepal
3550,Wine Consumption [l] in 2009,3.3,2.28,1.68,3.47,3.01,3.28,3.66,4.86,0.0,...,0.0,0.73,0.01,0.01,3.85,0.33,0.0,1.31,0.01,0.0
3551,Wine Consumption [l] in 2010,3.4,2.29,1.70,2.26,3.12,3.39,3.71,4.86,0.0,...,0.0,0.75,0.00,0.02,3.72,0.46,0.0,1.32,0.01,0.0
3552,Wine Consumption [l] in 2011,3.6,2.39,1.74,2.32,3.20,3.14,3.71,5.05,0.0,...,0.0,0.59,0.00,0.02,3.43,0.54,0.0,1.09,0.01,0.0
3553,Wine Consumption [l] in 2012,3.6,2.29,1.70,2.30,3.19,3.16,3.73,4.95,0.0,...,0.0,0.00,0.00,0.05,3.18,0.00,0.0,0.73,0.00,0.0
3554,Wine Consumption [l] in 2013,3.6,2.30,1.71,2.04,3.08,2.97,4.26,4.79,0.0,...,0.0,0.00,0.00,0.05,3.25,0.00,0.0,0.53,0.00,0.0
3555,Wine Consumption [l] in 2014,3.6,2.25,1.67,2.03,3.10,0.00,4.53,4.75,0.0,...,0.0,0.00,0.00,0.00,3.13,0.00,0.0,0.00,0.00,0.0
3556,Wine Consumption [l] in 2015,0.0,2.22,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
3557,3557,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
3558,3558,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
3559,3559,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0


(3557, 197)

In [19]:
gdp = 'maddison-data-gdp-per-capita-in-2011us.csv'
df = pd.read_csv(os.path.join(datapath, gdp ) , encoding='latin-1')
print(df.shape)
result_df = df[df["Year"] > 1979]
result_df

(5862, 6)


,Unnamed: 0,Unnamed: 0.1,Entity,Code,Year,GDP per capita [$]
0,30,30,Afghanistan,AFG,1980,2108
1,31,31,Afghanistan,AFG,1981,2369
2,32,32,Afghanistan,AFG,1982,2628
3,33,33,Afghanistan,AFG,1983,2789
4,34,34,Afghanistan,AFG,1984,2766
5,35,35,Afghanistan,AFG,1985,2700
6,36,36,Afghanistan,AFG,1986,2779
7,37,37,Afghanistan,AFG,1987,2507
8,38,38,Afghanistan,AFG,1988,2280
9,39,39,Afghanistan,AFG,1989,2067


In [12]:
## VALIDATE BEER AND WINE CONSUMPTION
vege = 'vegetable-consumption-per-capita.csv'
wine = 'wine-consumption-per-person.csv'
beer = 'beer-consumption-per-person.csv'


test_df = pd.DataFrame(data = np.zeros( (500, len(interested_countries)) ), 
                        index = np.arange(500),
                        columns = [ country for country in interested_countries])
# save indices as strings to get meaningful names
test_df.index = test_df.index.map(str)


# paste the values into the final_df from each single list
listies = [vege, wine, beer]
nextListIndex = 0

for lists in listies:

    ## Read the file and get the all countries along with their reported years
    df = pd.read_csv(os.path.join(datapath, str(lists) ) , encoding='latin-1')

    # get the col names, unique countries and unique years
    columns = list(df.columns)
    countries = list(set(df["Entity"]))
    years = list(set(df['Year']))

    # get a list of all the index/row names
    indexNamesArr = test_df.index.values

    ## groupby countries and then years accordingly
    #df.groupby(["Entity", 'Year'])

    firstListRun = False # flag for renaming the indices

    # loop thru every country in the list
    for country in countries:

        ## check if country is in our masterlist
        if country not in interested_countries: 
            # skip this item
            print("\tCountry: {} not in list - but we skip it.".format(country))
            continue

        # take a dataframe for one country at a time
        country_df = df[df['Entity'] == country]

        # loop thru every year within that country - assuming the years are in the same order for every country
        for year in years:

            # rename the indices only if it is the very first run for the country
            if not firstListRun:
                indexName = str(columns[-1]) + ' in ' + str(year)
                indexNamesArr[nextListIndex] = indexName
                nextListIndex += 1
                if nextListIndex % 20 == 0: 
                    print("\t\t" + str(indexName))

            ## get the proper value and fill empty ones, if not available, fill it with 0.000
            # note: .sum() is only having one element anyway, just done to get the value as a float not an array
            value = country_df[country_df["Year"] == year][columns[-1]].sum() if not country_df[country_df["Year"] == year][columns[-1]].empty else 0.000
            # get the name of the row
            idxName = str(columns[-1]) + ' in ' + str(year)

            #print("Country: {} found in the dataset at spot: {}".format(country, interested_countries.index(country)))

            # assign the value in the merged df with the value 
            test_df.iat[list(test_df.index.values).index(idxName), interested_countries.index(country)] = value # .iat[row, col]

        ## get the proper index for the next list to begin with   
        firstListRun = True
        # print progress
        if countries.index(country) % 60 == 0:
            print("\tWorking on country: {} out of {}".format(countries.index(country), len(countries)))


		Vegetables per capita [kg] in 1980
		Vegetables per capita [kg] in 2000
	Working on country: 0 out of 161
	Working on country: 60 out of 161
	Working on country: 120 out of 161
		Wine Consumption [l] in 1966
		Wine Consumption [l] in 1986
		Wine Consumption [l] in 2006
	Working on country: 0 out of 173
	Working on country: 60 out of 173
	Working on country: 120 out of 173
		Beer Consumption per capita [l] in 1970
		Beer Consumption per capita [l] in 1990
		Beer Consumption per capita [l] in 2010
	Working on country: 0 out of 173
	Working on country: 60 out of 173
	Working on country: 120 out of 173


In [15]:
test_df[100:150]

,Sweden,Norway,Finland,Iceland,Germany,Netherlands,Belgium,Luxembourg,England,Scotland,...,Cayman Islands,Botswana,Ethiopia,Chad,Uruguay,Maldives,Gibraltar,Paraguay,Niger,Nepal
Wine Consumption [l] in 2007,3.10,2.11,1.64,2.04,3.14,3.18,4.27,5.07,0.0,0.0,...,0.0,1.86,0.01,0.00,4.15,0.37,0.0,1.35,0.01,0.00
Wine Consumption [l] in 2008,3.10,2.23,1.67,2.42,3.13,3.29,3.83,4.92,0.0,0.0,...,0.0,1.16,0.00,0.00,3.97,0.37,0.0,1.74,0.01,0.00
Wine Consumption [l] in 2009,3.30,2.28,1.68,3.47,3.01,3.28,3.66,4.86,0.0,0.0,...,0.0,0.73,0.01,0.01,3.85,0.33,0.0,1.31,0.01,0.00
Wine Consumption [l] in 2010,3.40,2.29,1.70,2.26,3.12,3.39,3.71,4.86,0.0,0.0,...,0.0,0.75,0.00,0.02,3.72,0.46,0.0,1.32,0.01,0.00
Wine Consumption [l] in 2011,3.60,2.39,1.74,2.32,3.20,3.14,3.71,5.05,0.0,0.0,...,0.0,0.59,0.00,0.02,3.43,0.54,0.0,1.09,0.01,0.00
Wine Consumption [l] in 2012,3.60,2.29,1.70,2.30,3.19,3.16,3.73,4.95,0.0,0.0,...,0.0,0.00,0.00,0.05,3.18,0.00,0.0,0.73,0.00,0.00
Wine Consumption [l] in 2013,3.60,2.30,1.71,2.04,3.08,2.97,4.26,4.79,0.0,0.0,...,0.0,0.00,0.00,0.05,3.25,0.00,0.0,0.53,0.00,0.00
Wine Consumption [l] in 2014,3.60,2.25,1.67,2.03,3.10,0.00,4.53,4.75,0.0,0.0,...,0.0,0.00,0.00,0.00,3.13,0.00,0.0,0.00,0.00,0.00
Wine Consumption [l] in 2015,0.00,2.22,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00
Beer Consumption per capita [l] in 1960,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00


Comparing the above dataframe with the real lists from the UN, we see that they do match, although the wine consumption seems a little suspisous to me to be honest. <br /> 
By [checking this guy here](https://ourworldindata.org/grapher/wine-consumption-per-person) again, they mention that they only record the total alcohol amount of wine. Wine usually has 12% of alcohol in a bottle, thus 1l of wine contains 0.12l of pure alcohol in it. Thus 3l of pure alcohol from wine is the equvalent of approximately 25 bottles of wine. <br /> 

Hence, the data is accordingly sorted in the right columns and rows and we can go ahead and further analyse this fresh gut here then! 

In [ ]:
df.groupby('Entity')['Wine Consumption'].sum().sort_values().tail(5)

In [ ]:
df[df['Entity'] == 'Germany']['Wine Consumption'].plot.hist(bins=20)